In [13]:
import pandas as pd
import os
import gc
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler
from gensim.models import Word2Vec
import math
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

Slow version of gensim.models.doc2vec is being used
Slow version of Fasttext is being used


In [ ]:
ent_default=pd.read_csv('/home/c506plus/wq/AIWIN/国泰/2022S-T3-1st-Training/ent_default.csv',delimiter="|")
ent_info=pd.read_csv('/home/c506plus/wq/AIWIN/国泰/2022S-T3-1st-Training/ent_info.csv',delimiter="|")
ent_financial=pd.read_csv('/home/c506plus/wq/AIWIN/国泰/2022S-T3-1st-Training/ent_financial_indicator.csv',delimiter="|")
ent_news=pd.read_csv('/home/c506plus/wq/AIWIN/国泰/2022S-T3-1st-Training/ent_news.csv',delimiter="|",error_bad_lines=False)
sample=pd.read_csv('/home/c506plus/wq/AIWIN/国泰/sample_submission/answer.csv',delimiter="|")

In [4]:
news_list = []
for idx, line in enumerate(open('/home/c506plus/wq/AIWIN/国泰/2022S-T3-1st-Training/ent_news.csv', encoding='utf-8')):
    if idx == 0:
        cols = line.split('|')
    else:
        line_list = line.split('|')
        line_list = line_list[:8] + [''.join(line_list[8:]).replace('\n','')]
        news_list.append(line_list)
news_df = pd.DataFrame(news_list, columns=cols)

ent_default = pd.read_csv('/home/c506plus/wq/AIWIN/国泰/2022S-T3-1st-Training/ent_default.csv', sep='|')
ent_fina = pd.read_csv('/home/c506plus/wq/AIWIN/国泰/2022S-T3-1st-Training/ent_financial_indicator.csv', sep='|')
ent_info = pd.read_csv('/home/c506plus/wq/AIWIN/国泰/2022S-T3-1st-Training/ent_info.csv', sep='|')
answer = pd.read_csv('/home/c506plus/wq/AIWIN/国泰/sample_submission/answer.csv', sep='|')

### 时间处理
ent_default['year'] = ent_default['acu_date'].apply(lambda x:x//10000)
ent_fina['year'] = ent_fina['report_period'].apply(lambda x:x//10000)
news_df['year'] = news_df['publishdate'].apply(lambda x:int(x)//10000)

ent_default['ent_id_year'] = ent_default['ent_id'] + '_' + (ent_default['year'] - 1).astype(str)
ent_fina['ent_id_year'] = ent_fina['ent_id'] + '_' + ent_fina['year'].astype(str)
news_df['ent_id_year'] = news_df['ent_id'] + '_' + news_df['year'].astype(str)
answer['ent_id_year'] = answer['ent_id'].apply(lambda x: x+'_2020')

del ent_fina['year'], news_df['year']


# 去重
ent_default_new = ent_default.drop_duplicates(subset=['ent_id_year'], keep='last')

# 合并
ent_default_new['default_score'] = 1
answer['year'] = 2021
data = pd.concat([ent_default_new[['ent_id','ent_id_year','year','default_score']], answer], axis=0, ignore_index=True)
del ent_default_new

/home/c506plus/anaconda3/envs/sohu/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
ent_ids = [i for i in answer['ent_id'].unique() if i not in ent_default[ent_default['year']==2019]['ent_id'].unique().tolist()]
ent_ids_df = pd.DataFrame({'ent_id':ent_ids})
ent_ids_df['year'] = 2019
ent_ids_df['default_score'] = 0
ent_ids_df['ent_id_year'] = ent_ids_df['ent_id'].apply(lambda x: x+'_2018')
data = pd.concat([data, ent_ids_df], axis=0, ignore_index=True)

ent_ids = [i for i in answer['ent_id'].unique() if i not in ent_default[ent_default['year']==2020]['ent_id'].unique().tolist()]
ent_ids_df = pd.DataFrame({'ent_id':ent_ids})
ent_ids_df['year'] = 2020
ent_ids_df['default_score'] = 0
ent_ids_df['ent_id_year'] = ent_ids_df['ent_id'].apply(lambda x: x+'_2019')
data = pd.concat([data, ent_ids_df], axis=0, ignore_index=True)

In [8]:
data

,ent_id,ent_id_year,year,default_score
0,q00350f4fa8694d82639aec1653cf0306,q00350f4fa8694d82639aec1653cf0306_2018,2019,1
1,q005d1b175fed3b08e45aa7a402ac0072,q005d1b175fed3b08e45aa7a402ac0072_2018,2019,1
2,q01e01a62678f34dacfaa178d8866b7b0,q01e01a62678f34dacfaa178d8866b7b0_2018,2019,1
3,q0485d846345eabfa7e3fa4455c9e7a70,q0485d846345eabfa7e3fa4455c9e7a70_2018,2019,1
4,q04c0c103124d53d281131049f323b302,q04c0c103124d53d281131049f323b302_2019,2020,1
...,...,...,...,...
26921,qffd9dfb83fe140b08edda7cdde0907dc,qffd9dfb83fe140b08edda7cdde0907dc_2019,2020,0
26922,qffde2f552f0eb8df2f638ca79f99ca5f,qffde2f552f0eb8df2f638ca79f99ca5f_2019,2020,0
26923,qffe08330180da5aa66ba5387e0cb69de,qffe08330180da5aa66ba5387e0cb69de_2019,2020,0
26924,qffea6b9d34f25b9fc4820d84fb1f08d0,qffea6b9d34f25b9fc4820d84fb1f08d0_2019,2020,0


In [9]:
# ent_info.csv(企业的基本信息)
ent_info_cat_cols = ['industryphy','industryco','enttype','entstatus','prov','city','county','is_bondissuer'] 
ent_info_num_cols = ['regcap']
ent_info_time_cols = ['opfrom','opto','esdate','apprdate'] # 经营(驻在)期限自、经营(驻在)期限至、成立日期、核准日期

# ent_financial_indicator.csv(企业的财务指标数据)
ent_fina_num_cols = [f for f in ent_fina.columns if f not in ['ent_id','report_period','ent_id_year']]
ent_fina_time_cols = ['report_period'] # 报告期

# 自然数编码
def label_encode(series):
    unique = list(series.unique())
    # unique.sort()
    return series.map(dict(zip(unique, range(series.nunique()))))

for col in ent_info_cat_cols:
    ent_info[col] = label_encode(ent_info[col])
    
# 合并ent_info、ent_fina
print(data.shape)
ent_info_new = ent_info.drop_duplicates()
data = data.merge(ent_info_new, on=['ent_id'], how='left')

print(data.shape)

ent_fina_new = ent_fina.sort_values('report_period').drop_duplicates(subset=['ent_id_year'], keep='last')
data = data.merge(ent_fina_new, on=['ent_id','ent_id_year'], how='left')
print(data.shape)

(26926, 4)
(26926, 17)
(26926, 180)


In [10]:
data

,ent_id,ent_id_year,year,default_score,industryphy,industryco,opfrom,opto,esdate,apprdate,...,s_qfa_yoysales,s_qfa_cgrsales,s_qfa_yoyop,s_qfa_cgrop,s_qfa_yoyprofit,s_qfa_cgrprofit,s_qfa_yoynetprofit,s_qfa_cgrnetprofit,s_fa_yoy_equity,rd_expense
0,q00350f4fa8694d82639aec1653cf0306,q00350f4fa8694d82639aec1653cf0306_2018,2019,1,5.0,76.0,2010-05-11,2030-05-10,2010-05-11,2014-12-12,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-55.7344,0.000000e+00
1,q005d1b175fed3b08e45aa7a402ac0072,q005d1b175fed3b08e45aa7a402ac0072_2018,2019,1,0.0,191.0,1998-08-25,2048-08-25,1998-08-25,2022-03-09,...,9.3287,18.1813,-376.4088,-284.9769,-331.8106,-367.2886,3.3798,-245.5373,-40.8727,0.000000e+00
2,q01e01a62678f34dacfaa178d8866b7b0,q01e01a62678f34dacfaa178d8866b7b0_2018,2019,1,4.0,189.0,1993-06-14,9999-12-31,1993-06-14,2018-01-26,...,-131.9653,-159.1634,-5217.9072,-15728.1518,-7450.4862,-21439.8637,-7336.9729,-21436.0190,-136.3506,0.000000e+00
3,q0485d846345eabfa7e3fa4455c9e7a70,q0485d846345eabfa7e3fa4455c9e7a70_2018,2019,1,6.0,379.0,1993-08-14,9999-12-31,1993-08-14,2018-09-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,q04c0c103124d53d281131049f323b302,q04c0c103124d53d281131049f323b302_2019,2020,1,13.0,418.0,1999-04-28,2026-08-24,1999-04-28,2021-12-31,...,-23.1288,75.6115,-969.3446,-1051.3499,-1145.8558,-1196.9734,-717.5617,-1068.8059,2837.6078,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26921,qffd9dfb83fe140b08edda7cdde0907dc,qffd9dfb83fe140b08edda7cdde0907dc_2019,2020,0,4.0,539.0,2008-06-20,NaN,2008-06-20,2021-09-29,...,28.5060,23.3694,24.2987,10.1983,17.3949,12.2150,16.5176,13.7361,11.0760,2.987402e+07
26922,qffde2f552f0eb8df2f638ca79f99ca5f,qffde2f552f0eb8df2f638ca79f99ca5f_2019,2020,0,6.0,322.0,1994-05-03,2044-05-03,1994-05-03,2017-12-25,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.3008,0.000000e+00
26923,qffe08330180da5aa66ba5387e0cb69de,qffe08330180da5aa66ba5387e0cb69de_2019,2020,0,0.0,0.0,2003-07-08,9999-09-09,2003-07-08,2021-10-28,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,3.1592,0.000000e+00
26924,qffea6b9d34f25b9fc4820d84fb1f08d0,qffea6b9d34f25b9fc4820d84fb1f08d0_2019,2020,0,2.0,3.0,1999-10-19,NaN,1999-10-19,2021-11-26,...,10.4126,0.0000,347.9617,1423.8317,34.8058,1291.6281,34.8058,1291.6281,10.2684,0.000000e+00


In [11]:
features = [f for f in data.columns if f not in ['ent_id','ent_id_year','default_score','is_bondissuer'] + \
                                                ent_info_time_cols + ent_fina_time_cols]

train = data[data.year!=2021].reset_index(drop=True)
test = data[data.year==2021].reset_index(drop=True)

x_train = train[features]
x_test = test[features]

y_train = train['default_score']

In [14]:
def cv_model(clf, train_x, train_y, test_x, clf_name):
    folds = 5
    seed = 2022
    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 5,
                'num_leaves': 2 ** 5,
                'lambda_l2': 10,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.01,
                'seed': 2022,
                'n_jobs':-1,
                'verbose': -1,
            }

            model = clf.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], 
                              categorical_feature=[], verbose_eval=500, early_stopping_rounds=500)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
            
            # print(list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])
            
        train[valid_index] = val_pred
        test += test_pred / kf.n_splits
        cv_scores.append(roc_auc_score(val_y, val_pred))
        
        print(cv_scores)
       
    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    return train, test
    
def lgb_model(x_train, y_train, x_test):
    lgb_train, lgb_test = cv_model(lgb, x_train, y_train, x_test, "lgb")
    return lgb_train, lgb_test
    
lgb_train, lgb_test = lgb_model(x_train, y_train, x_test)

************************************ 1 ************************************
Training until validation scores don't improve for 500 rounds
[500]	training's auc: 0.979015	valid_1's auc: 0.945491
[1000]	training's auc: 0.988989	valid_1's auc: 0.950173
[1500]	training's auc: 0.993245	valid_1's auc: 0.95231
[2000]	training's auc: 0.99579	valid_1's auc: 0.953922
[2500]	training's auc: 0.997349	valid_1's auc: 0.953116
Early stopping, best iteration is:
[2289]	training's auc: 0.996766	valid_1's auc: 0.954321
[0.9543206969625618]
************************************ 2 ************************************
Training until validation scores don't improve for 500 rounds
[500]	training's auc: 0.978808	valid_1's auc: 0.93397
[1000]	training's auc: 0.989057	valid_1's auc: 0.947722
[1500]	training's auc: 0.994123	valid_1's auc: 0.948498
[2000]	training's auc: 0.996725	valid_1's auc: 0.950519
[2500]	training's auc: 0.998096	valid_1's auc: 0.951043
[3000]	training's auc: 0.998792	valid_1's auc: 0.951698
[

In [15]:
test['default_score'] = lgb_test
answer = pd.merge(answer[['ent_id']], test[['ent_id','default_score']], on=['ent_id'], how='left')
answer[['ent_id', 'default_score']].to_csv('4_29_1.csv', header=True, index=False, sep='|')